In [ ]:
using MuJoCo
using LinearAlgebra
using MatrixEquations  
import MuJoCo as MJ

model = MJ.load_model("../unitree_robots/g1/scene_29dof.xml")      #
data  = MJ.init_data(model)               

MJ.init_visualiser()

In [ ]:
# default upright pose from XML (might have to chage to one leg pose)
reset!(model, data)
forward!(model, data)
data.qacc .= 0

# inverse dynamics to get required torques for static pose
MJ.mj_inverse(model, data)
qpos0 = copy(data.qpos)
qfrc0 = copy(data.qfrc_inverse)

# generalized forces to actuator controls
M_act = data.actuator_moment
ctrl0 = pinv(M_act)' * qfrc0

29×1 Matrix{Float64}:
 -1.2192965303060772
 -0.06826768606121636
  0.009351820161128899
 -0.25623776897999995
 -0.15281168454
  0.0
 -1.2192965303060772
  0.06826768606121814
 -0.00935182016112912
 -0.25623776897999995
  ⋮
 -0.40097180970350166
  1.0381295464721673e-5
 -2.0933910619160376
 -0.19408874489754613
 -0.00012846741010276875
 -1.8814202856289333
  0.004373843135833999
 -0.40097180970350166
 -1.0381295464721673e-5

In [ ]:
nu = model.nu
nv = model.nv

# LQR cost matrices
R = Matrix{Float64}(I, nu, nu)

free_dofs = 1:6
body_dofs = 7:nv
waist_dofs = [j.id+1 for j in MJ.joints(model) if occursin("waist", j.name)]
left_leg_dofs = [j.id+1 for j in MJ.joints(model) if occursin("left_", j.name) && any(occursin(p, j.name) for p in ("hip","knee","ankle"))]
balance_dofs = vcat(waist_dofs, left_leg_dofs)
other_dofs = setdiff(body_dofs, balance_dofs)

balance_cost = 1000
balance_joint_cost = 3
other_joint_cost = 0.3

0.3

In [24]:
# Compute Jacobians
reset!(model, data)
data.qpos .= qpos0
forward!(model, data)

jac_com = MJ.mj_zeros(3, nv)
pelvis = MJ.body(model, "pelvis")
MJ.mj_jacSubtreeCom(model, data, jac_com, pelvis.id)

jac_foot = MJ.mj_zeros(3, nv)
foot_body = MJ.body(model, "left_ankle_roll_link")
MJ.mj_jacBodyCom(model, data, jac_foot, nothing, foot_body.id)

jac_diff = jac_com .- jac_foot
Qbalance = jac_diff' * jac_diff

Qjoint = Matrix{Float64}(I, nv, nv)
Qjoint[free_dofs, free_dofs] .= 0
Qjoint[balance_dofs, balance_dofs] .*= balance_joint_cost
Qjoint[other_dofs, other_dofs] .*= other_joint_cost

Qpos = balance_cost * Qbalance + Qjoint
Q = Matrix{Float64}(I, 2*nv, 2*nv) * 1e-10
Q[1:nv, 1:nv] .+= Qpos

41×41 view(::Matrix{Float64}, 1:41, 1:41) with eltype Float64:
  1.0e-10       0.0           0.0          …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0           1.0e-10       0.0             0.0  0.0  0.0  0.0  0.0  0.0
  0.0           0.0           1.0e-10         0.0  0.0  0.0  0.0  0.0  0.0
  0.0          -3.11795e-13  -5.25954e-14     0.0  0.0  0.0  0.0  0.0  0.0
  3.11795e-13   0.0           3.07849e-15     0.0  0.0  0.0  0.0  0.0  0.0
  5.25954e-14  -3.07849e-15   0.0          …  0.0  0.0  0.0  0.0  0.0  0.0
  2.71683e-13   0.0           1.01975e-14     0.0  0.0  0.0  0.0  0.0  0.0
  1.74312e-16  -2.5865e-13   -9.86458e-16     0.0  0.0  0.0  0.0  0.0  0.0
  9.66675e-16   4.44667e-14   1.70817e-16     0.0  0.0  0.0  0.0  0.0  0.0
  1.42534e-13   0.0           1.14402e-14     0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                                        ⋱  ⋮                        ⋮
 -2.02367e-24   1.62787e-18   5.63906e-18     0.0  0.0  0.0  0.0  0.0  0.0
 -1.65647e-18  -3.05822e-20  -5.16961e-

In [ ]:
# Linearize dynamics around upright pose
data.qpos .= qpos0
data.qvel .= 0
data.ctrl .= ctrl0

A = MJ.mj_zeros(2*nv, 2*nv)
B = MJ.mj_zeros(2*nv, nu)

ϵ = 1e-6
MJ.mjd_transitionFD(model, data, ϵ, true, A, B, nothing, nothing)

# Reduce to actuated joints only (exclude floating base)
actuated = 7:nv

A_joint = [A[actuated, actuated] A[actuated, nv .+ actuated];
           A[nv .+ actuated, actuated] A[nv .+ actuated, nv .+ actuated]]
B_joint = [B[actuated, :]; B[nv .+ actuated, :]]

Q_joint = Q[ [actuated; nv .+ actuated], [actuated; nv .+ actuated] ]
R_joint = R
S_joint = zeros(size(Q_joint,1), size(R_joint,1))

# regularization to avoid ricatti failure
ϵ_qr = 1e-6
Q_joint .+= ϵ_qr * Matrix{Float64}(I, size(Q_joint)...)
R_joint .+= ϵ_qr * Matrix{Float64}(I, size(R_joint)...)

# Solve the riccati for joint-only subsystem
_, _, K_joint, _ = ared(A_joint, B_joint, R_joint, Q_joint, S_joint)


# Controller callback (uses joint-only state)
function g1_ctrl!(m::Model, d::Data)
    Δq = zeros(nv)
    MJ.mj_differentiatePos(m, Δq, 1.0, qpos0, d.qpos)
    Δx_joint = vcat(Δq[actuated], d.qvel[actuated])
    d.ctrl .= ctrl0 .- K_joint * Δx_joint
    return nothing
end

ErrorException: The extended symplectic pencil is not dichotomic

In [ ]:
reset!(model, data)
data.qpos .= qpos0
data.qvel .= 0
data.ctrl .= ctrl0
MJ.visualise!(model, data, controller = g1_ctrl!)